# Day 8: Resonant Collinearity

### Part 1

You find yourselves on the roof of a top-secret Easter Bunny installation.

While The Historians do their thing, you take a look at the familiar huge antenna. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand Imitation Mediocre Chocolate as a Christmas gift! Unthinkable!

Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific frequency indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:

```
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
```

The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency a, they create the two antinodes marked with #:

```
..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
```

Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
```

Antennas with different frequencies don't create antinodes; A and a count as different frequencies. However, antinodes can occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital A creates no antinodes but has a lowercase-a-frequency antinode at its location:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
```

The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost A-frequency antenna:

```
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
```

Because the topmost A-frequency antenna overlaps with a 0-frequency antinode, there are 14 total unique locations that contain an antinode within the bounds of the map.

Calculate the impact of the signal. How many unique locations within the bounds of the map contain an antinode?


In [1]:
import com.psyndicate.aoc.*
import kotlin.test.assertEquals

val testInput =
    """
    ............
    ........0...
    .....0......
    .......0....
    ....0.......
    ......A.....
    ............
    ............
    ........A...
    .........A..
    ............
    ............
    """.trimIndent()

data class Antenna(val frequency: Char, val location: Point)

fun Line.calculateAntiNodes(): Set<Point> {
    val c = Point(a.x - dX, a.y - dY)
    val d = Point(b.x + dX, b.y + dY)
    return setOf(c,d)
}


fun calculatePart1(input: String): Int {
    val map = bitmapFromString(input)
    val antenna = map.coords().mapNotNull {
        when (val c = map[it]) {
            '.' -> null
            null -> null
            else -> Antenna(c, it.toPoint())
        }
    }.groupBy { it.frequency }
    val nodes = antenna.keys.map { freqency ->
        freqency to (antenna[freqency]?.square { a, b ->
            if (a != b) Line(a.location, b.location).calculateAntiNodes() else emptySet()
        }
            ?.flatten()
            ?.filter{ map.isInBounds(it.toCoord())}
            ?.toSet()
            ?: emptySet())
    }.toMap()
    return nodes.values.flatten().distinct().count()
}

val testInputPart1Result = calculatePart1(testInput)
println("Test Part 1: ${testInputPart1Result}")
assertEquals(14, testInputPart1Result)

val puzzleInput = readResourceAsText("/day8/input.txt")
val part1Result = calculatePart1(puzzleInput)
println("Part 1: ${part1Result}")
assertEquals(367, part1Result)


Test Part 1: 14
Part 1: 367


### Part 2

Watching over your shoulder as you work, one of The Historians asks if you took the effects of resonant harmonics into your calculations.

Whoops!

After updating your model, it turns out that an antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).

So, these three T-frequency antennas now create many antinodes:

```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```

In fact, the three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to 9.

The original example now has 34 antinodes, including the antinodes that appear on every antenna:

```
##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
```

Calculate the impact of the signal using this updated model. How many unique locations within the bounds of the map contain an antinode?



In [10]:
fun Line.calculateAntiNodes(validCheck: (Point) -> Boolean): Set<Point> = buildSet {
    for(dist in 0..1000) {
        var valid = false
        val c = Point(a.x - dX * dist, a.y - dY * dist)
        if (validCheck(c)) {
            add(c)
            valid = true
        }

        val d = Point(b.x + dX * dist, b.y + dY * dist)
        if (validCheck(d)) {
            add(d)
            valid = true
        }
        if (!valid) return@buildSet
    }
}

fun calculatePart2(input: String): Int {
    val map = bitmapFromString(input)
    val antenna = map.coords().mapNotNull {
        when (val c = map[it]) {
            '.' -> null
            null -> null
            else -> Antenna(c, it.toPoint())
        }
    }.groupBy { it.frequency }
    val nodes = antenna.keys.map { freqency ->
        freqency to (antenna[freqency]?.square { a, b ->
            if (a != b) Line(a.location, b.location).calculateAntiNodes {
                map.isInBounds(it.toCoord())
            } else emptySet()
        }
            ?.flatten()
            ?.toSet()
            ?: emptySet())
    }.toMap()
    return nodes.values.flatten().distinct().count()
}

val testInputPart2Result = calculatePart2(testInput)
println("Test Part 2: ${testInputPart2Result}")
assertEquals(34, testInputPart2Result)

val part2Result = calculatePart2(puzzleInput)
println("Part 2: ${part2Result}")
assertEquals(1285, part2Result)

Test Part 2: 34
Part 2: 1285
